In [ ]:
import panel as pn
pn.extension()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
import param


pn.extension()

class InteractiveFunction(param.Parameterized):
    n = param.Integer(default=5, bounds=(1, 20), doc="Number of rows in the input matrix")
    size = param.Integer(default=50, bounds=(40, 500), doc="Parameter size")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.input_fields = []
        self.output_labels = []
        self.plot_pane = pn.pane.Matplotlib()
        self.param.watch(self.update_plot, ['n', 'size'])
        self.create_inputs()
    
    def create_inputs(self):
        prop_names = [f'Size [\u03BCm]', '\u03BC\u2090 [mm\u207B\u00B9]', '\u03BC\u209B [mm\u207B\u00B9]', 'g', 'n']
        self.input_fields = [
            [pn.widgets.FloatInput(name=f'{prop_names[j]} - Layer {i+1}', value=0.0, width=110) for j in range(5)]
            for i in range(self.n)
        ]
        self.output_labels = [pn.pane.Markdown(f"Sum: 0") for _ in range(self.n)]
    
    def update_plot(self, *events):
        self.create_inputs()
        values = np.array([[widget.value for widget in row] for row in self.input_fields])
        sums = values.sum(axis=1)
        
        for i, row_sum in enumerate(sums):
            self.output_labels[i].object = f"Sum: {row_sum:.2f}"
        
        fig, ax = plt.subplots()
        fig.suptitle('Preview of the Digital Phantom')
        ax.imshow(values, cmap='viridis', aspect='auto')
        plt.colorbar(ax.imshow(values, cmap='viridis', aspect='auto'))
        self.plot_pane.object = fig
    
    @param.depends('n', 'size')
    def view(self):
        n_slider = pn.widgets.IntSlider.from_param(self.param.n, name='Number of Layers')
        size_slider = pn.widgets.IntSlider.from_param(self.param.size, name='Phantom Size (in pixels)')
        inputs = [pn.Row(*row, self.output_labels[i]) for i, row in enumerate(self.input_fields)]
        buttons = pn.Row(
            pn.widgets.Button(name="Run the Simulation", button_type="primary"),
            pn.widgets.Button(name="More Information", button_type="default")
        )
        return pn.Column(
            n_slider,
            size_slider,
            *inputs,
            buttons,
            self.plot_pane
        )

interactive_function = InteractiveFunction()
interactive_function.view().servable()
